### 1. Hava durumu modeli¶
Bu ödev için, geçmiş sıcaklık veri kümesine yeniden bakacaksınız. Aşağıdaki görevler için çözümünüzü içeren Jupyter notebooks'u gist dosyasına link olarak ekleyin.

Öncelikle, Sıcaklık verilerini Kaggle'dan indirin.
Önceki derste olduğu gibi, hedef değişkeninizin görünür sıcaklık ve sıcaklık arasındaki fark olduğu bir doğrusal regresyon modeli oluşturun. Açıklayıcı değişkenler olarak nem ve rüzgar hızı kullanın. Şimdi, modelinizi OLS kullanarak tahmin edin. R-kare ve ayarlanmış R-kare değerleri nelerdir? Tatminkar olduklarını düşünüyor musunuz? Açıklayın.

Daha sonra, yukarıdaki modele nem ve rüzgar hızı etkileşimini dahil edin ve OLS'yi kullanarak modeli tahmin edin. Şimdi, bu modelin R-kare değeri nedir? Bu model bir öncekine göre gelişti mi?

İlk modele ek açıklayıcı değişken olarak görünürlük ekleyin ve tahmin edin. R-kare arttı mı? Ayarlanmış R-kare değeri ne oldu? Tabloda ortaya çıkan farklılıkları, ayarlanmış R-kare içindeki iyileşme açısından etkileşim terimi ve görünürlük ile karşılaştırın. Hangisi daha kullanışlı?

AIC ve BIC puanlarına göre yukarıdaki üç modelden en iyisini seçin. Mentor ile gerekçenizi tartışarak seçiminizi doğrulayın.

In [317]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale

import warnings
warnings.filterwarnings('ignore')

In [318]:
tem_df = pd.read_csv("weatherHistory.csv")
tem_df.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [319]:
# değişken değerlerini standartlaştıralım.
names=tem_df.select_dtypes(include=["float","integer"]).columns
scaled_df = scale(tem_df[names])
scaled_df = pd.DataFrame(scaled_df,columns=names)

#### 1.model

In [320]:
Y = scaled_df["Apparent Temperature (C)"] - scaled_df["Temperature (C)"]
X = scaled_df[["Humidity","Wind Speed (km/h)"]]

X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                 2.145e+04
Date:                Fri, 23 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:12:46   Log-Likelihood:                 84256.
No. Observations:               96453   AIC:                        -1.685e+05
Df Residuals:                   96450   BIC:                        -1.685e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -4.109e-17      0.000  -1.26e-13      1.000      -0.001       0.001
Humidity              0.0157      0.000     47.094      0.000       0.015       0.016
Wind Speed (km/h)    -0.0621      0.000   -185.940      0.000      -0.063      -0.061
==============================================================================
Omnibus:                     1250.077   Durbin-Watson:                   0.427
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1354.445
Skew:                           0.253   Prob(JB):                    7.69e-295
Kurtosis:                       3.283   Cond. No.                         1.26
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-squared:0.308

Adj. R-squared:0.308  

Modeldeki bağımsız değişkenlerin, bağımlı değişkeni ne kadar açıkladığı R square değeri ile ölçülebilir.R square değerleri modelin hedef değişkenindeki varyansları açıklama oranını gösterir.Bu modelde varyansın %30.8 lik oranı açıklanmıs olup iyi bir oran değildir.Hedef değişkeni açıklamada yetersizdir diyebiliriz.

#### 2.Model

In [321]:
scaled_df["hum_wind_speed"] = scaled_df["Humidity"]*scaled_df["Wind Speed (km/h)"]

Y = scaled_df["Apparent Temperature (C)"]-scaled_df["Temperature (C)"] 
X = scaled_df[["Humidity","Wind Speed (km/h)","hum_wind_speed"]]

X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.407
Method:                 Least Squares   F-statistic:                 2.209e+04
Date:                Fri, 23 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:12:48   Log-Likelihood:                 91730.
No. Observations:               96453   AIC:                        -1.835e+05
Df Residuals:                   96449   BIC:                        -1.834e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0089      0.000    -28.819      0.000      -0.010      -0.008
Humidity              0.0156      0.000     50.554      0.000       0.015       0.016
Wind Speed (km/h)    -0.0679      0.000   -217.454      0.000      -0.069      -0.067
hum_wind_speed       -0.0396      0.000   -127.145      0.000      -0.040      -0.039
==============================================================================
Omnibus:                     3347.879   Durbin-Watson:                   0.453
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4857.732
Skew:                           0.353   Prob(JB):                         0.00
Kurtosis:                       3.844   Cond. No.                         1.37
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-squared:0.407
    
Adj. R-squared:0.407

Bu modelde hedef değişkenin varyansını açıklama oranımız %40.7 oldu. İlk modele göre R2 değeri arttı, bu da gösteriyor ki modele eklediğimiz humidity ve wind speed değişkenlerinin etkileşimi hedef değişkenin daha önce açıklanmayan bir varyansını açıkladı.

#### 3.Model

şimdi de ilk modele visibilty değişkenini ekleyerek tekrar deneyelim.

In [322]:
Y = scaled_df["Apparent Temperature (C)"] - scaled_df["Temperature (C)"]
X = scaled_df[["Humidity","Wind Speed (km/h)","Visibility (km)"]]

X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                 1.431e+04
Date:                Fri, 23 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:12:51   Log-Likelihood:                 84261.
No. Observations:               96453   AIC:                        -1.685e+05
Df Residuals:                   96449   BIC:                        -1.685e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -4.109e-17      0.000  -1.26e-13      1.000      -0.001       0.001
Humidity              0.0161      0.000     45.084      0.000       0.015       0.017
Wind Speed (km/h)    -0.0621      0.000   -185.973      0.000      -0.063      -0.061
Visibility (km)       0.0011      0.000      3.074      0.002       0.000       0.002
==============================================================================
Omnibus:                     1319.946   Durbin-Watson:                   0.428
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1431.816
Skew:                           0.262   Prob(JB):                    1.22e-311
Kurtosis:                       3.287   Cond. No.                         1.56
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3.modeli, humidity ve wind speed değişkenlerinin etkileşiminin yer aldığı 2.model ile karşılaştırırsak : 2.modelin R2 değeri daha yüksektir,hedef değişkeni daha iyi açıklamaktadır.Yani hum_wind_speed değişkeni Visibility değişkeninden daha yüksek oranda varyans açıklamıstır.Bu yüzden de 2.model daha kullanışlıdır.

AIC ve BIC puanlarına göre 3 modeli değerlendirirsek:
    
    1.model için:  AIC: -1.685e+05     BIC: -1.685e+05    R-squared: 0.308	      Adj. R-squared: 0.308

    2.model için:  AIC: -1.835e+05     BIC: -1.834e+05    R-squared: 0.407	      Adj. R-squared: 0.407

    3.model için:  AIC: -1.685e+05     BIC: -1.685e+05    R-squared: 0.308	      Adj. R-squared: 0.308

Hem AIC hem de BIC için, daha düşük değer daha iyidir. Bu nedenle, en düşük AIC veya BIC değeri olan modeli seçeriz.Yani bizim için en kullanışlı olan hem AIC,BIC kriterleri hem de R2 değerleri incelendiğinde 2.modeldir
	


### 2. Ev fiyatları modeli
Bu alıştırmada, ev fiyatları modelinizde çalışacaksınız. Aşağıdaki görevler için çözümünüzü içeren Jupyter notebooks'u gist dosyasına link olarak ekleyin.:

İlk olarak, Ev Fiyatları verilerini Kaggle'dan indirin.

Ev fiyatları modelinizi tekrar çalıştırın ve F-testi, R-kare, ayarlanmış R-kare, AIC ve BIC kullanarak modelinizin uygunluğunu değerlendirin.

Modelinizin tatmin edici olduğunu düşünüyor musunuz? Açıklayın

Modelinizin uygunluğunu iyileştirmek için, bazı değişkenleri ekleyerek veya kaldırarak farklı model özelliklerini deneyin.
Denediğiniz her model için, uygun metrikleri alın ve modellerinizi birbiriyle karşılaştırın. Hangi model en iyisidir ve neden?

In [323]:
pd.options.display.max_columns = None

house = pd.read_csv("train.csv")
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [324]:
#önce kategorik değişkenleri dönüştürerek baslayalım.
KitchenQual = {"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["KitchenQual"] = house["KitchenQual"].map(KitchenQual)

ExterQual = {"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["ExterQual"] = house["ExterQual"].map(ExterQual)

ExterCond = {"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["ExterCond"] = house["ExterCond"].map(ExterCond)

HeatingQC = {"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["HeatingQC"] = house["HeatingQC"].map(HeatingQC)

FireplaceQu = {"NA":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["FireplaceQu"] = house["FireplaceQu"].map(FireplaceQu)

In [325]:
#kategorik değişkenleri get_dummies ile dönüştürelim
get_dummies_house = pd.get_dummies(house,drop_first=True)


In [326]:
# sonra da değişken değerlerini standartlaştıralım.

house_names=get_dummies_house.select_dtypes(include=["float","integer"]).columns
house_scaled_df = scale(get_dummies_house[house_names])
house_scaled_df = pd.DataFrame(house_scaled_df,columns=house_names)

In [327]:
#SlePrice değişkeni ile en cok korelasyonu olan ilk 20 değişkene bakalım.
house_corr=house_scaled_df.corr().SalePrice.abs().sort_values(ascending=False).index[0:30]
house_scaled_df[house_corr].corr().SalePrice

SalePrice                1.000000
OverallQual              0.790982
GrLivArea                0.708624
ExterQual                0.682639
KitchenQual              0.659600
GarageCars               0.640409
GarageArea               0.623431
TotalBsmtSF              0.613581
1stFlrSF                 0.605852
FullBath                 0.560664
TotRmsAbvGrd             0.533723
YearBuilt                0.522897
YearRemodAdd             0.507101
Foundation_PConc         0.497734
GarageYrBlt              0.486362
MasVnrArea               0.477493
Fireplaces               0.466929
BsmtQual_TA             -0.452394
BsmtFinType1_GLQ         0.434597
HeatingQC                0.427649
GarageFinish_Unf        -0.410608
Neighborhood_NridgHt     0.402149
BsmtFinSF1               0.386420
MasVnrType_None         -0.374468
SaleType_New             0.357509
GarageType_Detchd       -0.354141
SaleCondition_Partial    0.352060
LotFrontage              0.351799
Foundation_CBlock       -0.343263
GarageType_Att

In [328]:
#SalePrice hedef değişkeni ile en yüksek collinearitye sahip olup aralarında multicollinearity 
#düşük olan değişkenler üzerinden model kuralım.

house_scaled_df[['OverallQual', 'GrLivArea', 'KitchenQual',
       'GarageCars', 'YearBuilt',  'Foundation_PConc',
        'Fireplaces','BsmtFinType1_GLQ', 'HeatingQC','Neighborhood_NridgHt',"TotalBsmtSF", 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd', 'SaleCondition_Partial', 'LotFrontage',
       'Foundation_CBlock', 'GarageType_Attchd']].corr() 

,OverallQual,GrLivArea,KitchenQual,GarageCars,YearBuilt,Foundation_PConc,Fireplaces,BsmtFinType1_GLQ,HeatingQC,Neighborhood_NridgHt,TotalBsmtSF,BsmtFinSF1,SaleType_New,GarageType_Detchd,SaleCondition_Partial,LotFrontage,Foundation_CBlock,GarageType_Attchd
OverallQual,1.000000,0.593007,0.673331,0.600671,0.572323,0.568211,0.396765,0.433448,0.457083,0.368724,0.537808,0.239666,0.327412,-0.337227,0.323295,0.251646,-0.429761,0.357994
GrLivArea,0.593007,1.000000,0.420563,0.467247,0.199010,0.292764,0.461679,0.193521,0.254644,0.179892,0.454868,0.208171,0.168368,-0.231550,0.163239,0.402797,-0.266787,0.140044
KitchenQual,0.673331,0.420563,1.000000,0.509810,0.530165,0.538126,0.256409,0.420482,0.504228,0.353777,0.432630,0.228415,0.337878,-0.268970,0.339578,0.181839,-0.400653,0.273187
GarageCars,0.600671,0.467247,0.509810,1.000000,0.537850,0.453421,0.300789,0.321159,0.325347,0.282752,0.434585,0.224054,0.286290,-0.153461,0.282164,0.285691,-0.318814,0.320615
YearBuilt,0.572323,0.199010,0.530165,0.537850,1.000000,0.651199,0.147716,0.479425,0.448855,0.268898,0.391452,0.249503,0.346954,-0.487130,0.343895,0.123349,-0.290574,0.472229
Foundation_PConc,0.568211,0.292764,0.538126,0.453421,0.651199,1.000000,0.103510,0.453762,0.529073,0.258332,0.307701,0.094711,0.328524,-0.292085,0.323299,0.061824,-0.781558,0.259788
Fireplaces,0.396765,0.461679,0.256409,0.300789,0.147716,0.103510,1.000000,0.114646,0.129689,0.151202,0.339519,0.260011,0.066107,-0.251002,0.065993,0.266639,-0.037848,0.248375
BsmtFinType1_GLQ,0.433448,0.193521,0.420482,0.321159,0.479425,0.453762,0.114646,1.000000,0.330643,0.189509,0.317396,0.497436,0.131802,-0.263651,0.131121,0.091015,-0.313389,0.268382
HeatingQC,0.457083,0.254644,0.504228,0.325347,0.448855,0.529073,0.129689,0.330643,1.000000,0.203891,0.265682,0.085672,0.248458,-0.223583,0.252279,0.098509,-0.427970,0.197427
Neighborhood_NridgHt,0.368724,0.179892,0.353777,0.282752,0.268898,0.258332,0.151202,0.189509,0.203891,1.000000,0.293445,0.131004,0.283105,-0.141707,0.278268,0.126684,-0.200541,0.081908


### Model 1

In [329]:
Y = house_scaled_df["SalePrice"]
X = house_scaled_df[['Foundation_PConc',
        'Fireplaces','BsmtFinType1_GLQ', "TotalBsmtSF",'HeatingQC','Neighborhood_NridgHt', 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd']]

X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.622
Model:                            OLS   Adj. R-squared:                  0.620
Method:                 Least Squares   F-statistic:                     265.2
Date:                Fri, 23 Apr 2021   Prob (F-statistic):          6.58e-299
Time:                        17:13:16   Log-Likelihood:                -1361.2
No. Observations:                1460   AIC:                             2742.
Df Residuals:                    1450   BIC:                             2795.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.301e-16      0.016   8.06e-15      1.000      -0.032       0.032
Foundation_PConc         0.1773      0.022      8.182      0.000       0.135       0.220
Fireplaces               0.2703      0.018     15.370      0.000       0.236       0.305
BsmtFinType1_GLQ         0.1180      0.021      5.495      0.000       0.076       0.160
TotalBsmtSF              0.2921      0.021     13.644      0.000       0.250       0.334
HeatingQC                0.1125      0.019      5.785      0.000       0.074       0.151
Neighborhood_NridgHt     0.1409      0.018      8.031      0.000       0.107       0.175
BsmtFinSF1               0.0462      0.022      2.103      0.036       0.003       0.089
SaleType_New             0.1118      0.018      6.270      0.000       0.077       0.147
GarageType_Detchd       -0.0410      0.018     -2.305      0.021      -0.076      -0.006
==============================================================================
Omnibus:                      552.097   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16754.252
Skew:                           1.131   Prob(JB):                         0.00
Kurtosis:                      19.441   Cond. No.                         3.22
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Model tatmin edici değil.Hedef deiğşken üzerinde etkili olduğunu düşündüğümüz değişkenlerden ekleyebiliriz.Öncesinde hataların dağılımına bakalım.

In [330]:
tahmin = results.predict(X)
hatalar = Y - tahmin
from scipy.stats import jarque_bera
from scipy.stats import normaltest
jb_stats = jarque_bera(hatalar)
norm_stats = normaltest(hatalar)

print("Jarque-Bera test değeri : {0} ve p değeri : {1}".format(jb_stats[0], jb_stats[1]))
print("Normal test değeri      : {0}  ve p değeri : {1:.30f}".format(norm_stats[0], norm_stats[1]))

Jarque-Bera test değeri : 16754.25239787975 ve p değeri : 0.0
Normal test değeri      : 552.096569747413  ve p değeri : 0.000000000000000000000000000000


pvalue < 0.05 olduğu için H0 hipotezi reddedilir ve hataların normal dağılmadığı kabul edilir.F testi hataların dağılımının normalliğine dayandığı için bunu incelemek model açısından önemlidir.hatalar normal dağılmadığından dolayı F testi bizi yanıltabilir.Spesifik olarak, OLS'nin bir doğrusal regresyon modeline uygulanması için hataların normalliği gerekmemektedir, fakat tahmin edilen katsayılarımızın istatistiksel anlamlılığını ölçmek için, hata terimleri normal olarak dağıtılmalıdır. 
Çoğu zaman, normal olarak dağıtılmamış hatalar ihmal edilen değişkenlerden kaynaklanır. Bu nedenle, modele ihmal edilen ilgili özellikleri eklemek sorunun düzeltilmesine yardımcı olabilir.

### Model 2

ilk modele ,hedef değişkenle ilişkisi olan 'GarageCars', 'YearBuilt' değişkenlerini modele ekleyip inceleyelim

In [331]:
Y = house_scaled_df["SalePrice"]
X = house_scaled_df[['GarageCars', 'YearBuilt',  'Foundation_PConc',
        'Fireplaces','BsmtFinType1_GLQ', "TotalBsmtSF",'HeatingQC','Neighborhood_NridgHt', 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd']]

X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     277.3
Date:                Fri, 23 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:13:21   Log-Likelihood:                -1244.2
No. Observations:                1460   AIC:                             2512.
Df Residuals:                    1448   BIC:                             2576.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.301e-16      0.015   8.73e-15      1.000      -0.029       0.029
GarageCars               0.3037      0.020     15.384      0.000       0.265       0.342
YearBuilt               -0.0353      0.024     -1.456      0.146      -0.083       0.012
Foundation_PConc         0.1078      0.022      4.873      0.000       0.064       0.151
Fireplaces               0.2121      0.017     12.666      0.000       0.179       0.245
BsmtFinType1_GLQ         0.0948      0.020      4.715      0.000       0.055       0.134
TotalBsmtSF              0.2223      0.020     10.970      0.000       0.183       0.262
HeatingQC                0.1008      0.018      5.584      0.000       0.065       0.136
Neighborhood_NridgHt     0.1187      0.016      7.291      0.000       0.087       0.151
BsmtFinSF1               0.0530      0.020      2.605      0.009       0.013       0.093
SaleType_New             0.0877      0.017      5.262      0.000       0.055       0.120
GarageType_Detchd       -0.0822      0.018     -4.551      0.000      -0.118      -0.047
==============================================================================
Omnibus:                      643.026   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17097.970
Skew:                           1.469   Prob(JB):                         0.00
Kurtosis:                      19.506   Cond. No.                         3.95
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Bu model de tatmin edici olmasa da ilk değişkene göre hedef değişkeni açıklayıcılık oranı arttı.

In [332]:
tahmin = results.predict(X)
hatalar = Y - tahmin
from scipy.stats import jarque_bera
from scipy.stats import normaltest
jb_stats = jarque_bera(hatalar)
norm_stats = normaltest(hatalar)

print("Jarque-Bera test değeri : {0} ve p değeri : {1}".format(jb_stats[0], jb_stats[1]))
print("Normal test değeri      : {0}  ve p değeri : {1:.30f}".format(norm_stats[0], norm_stats[1]))

Jarque-Bera test değeri : 17097.970052898345 ve p değeri : 0.0
Normal test değeri      : 643.0263815285418  ve p değeri : 0.000000000000000000000000000000


bu model için de hatalar normal dağılmıyor.

### Model 3

Modele etkisinin diğerlerinden daha cok olduğunu düşündüğümüz 'OverallQual', 'GrLivArea' değişkenleri ekleyip inceleyelim.

In [334]:
Y = house_scaled_df["SalePrice"]
X = house_scaled_df[['OverallQual', 'GrLivArea', 'KitchenQual',
       'GarageCars', 'YearBuilt', 'Fireplaces','BsmtFinType1_GLQ', "TotalBsmtSF",'HeatingQC','Neighborhood_NridgHt', 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd']]

X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.804
Method:                 Least Squares   F-statistic:                     462.5
Date:                Fri, 23 Apr 2021   Prob (F-statistic):               0.00
Time:                        17:13:30   Log-Likelihood:                -874.06
No. Observations:                1460   AIC:                             1776.
Df Residuals:                    1446   BIC:                             1850.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.301e-16      0.012   1.12e-14      1.000      -0.023       0.023
OverallQual              0.2514      0.021     12.132      0.000       0.211       0.292
GrLivArea                0.3146      0.016     19.318      0.000       0.283       0.347
KitchenQual              0.1139      0.017      6.626      0.000       0.080       0.148
GarageCars               0.1216      0.016      7.402      0.000       0.089       0.154
YearBuilt                0.0345      0.019      1.853      0.064      -0.002       0.071
Fireplaces               0.0668      0.014      4.837      0.000       0.040       0.094
BsmtFinType1_GLQ         0.0364      0.016      2.336      0.020       0.006       0.067
TotalBsmtSF              0.0705      0.017      4.268      0.000       0.038       0.103
HeatingQC                0.0304      0.014      2.159      0.031       0.003       0.058
Neighborhood_NridgHt     0.0882      0.013      6.847      0.000       0.063       0.114
BsmtFinSF1               0.1028      0.016      6.541      0.000       0.072       0.134
SaleType_New             0.0670      0.013      5.162      0.000       0.042       0.092
GarageType_Detchd       -0.0306      0.014     -2.160      0.031      -0.058      -0.003
==============================================================================
Omnibus:                      667.104   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           112513.004
Skew:                          -1.022   Prob(JB):                         0.00
Kurtosis:                      45.958   Cond. No.                         4.75
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [335]:
tahmin = results.predict(X)
hatalar = Y - tahmin

jb_stats = jarque_bera(hatalar)
norm_stats = normaltest(hatalar)

print("Jarque-Bera test değeri : {0} ve p değeri : {1}".format(jb_stats[0], jb_stats[1]))
print("Normal test değeri      : {0}  ve p değeri : {1:.30f}".format(norm_stats[0], norm_stats[1]))

Jarque-Bera test değeri : 112513.00415659651 ve p değeri : 0.0
Normal test değeri      : 667.1036520985524  ve p değeri : 0.000000000000000000000000000000


son model olan 3.model için de hatalar normal dağılmıyor.

3 model de incelendiğinde tüm değerler için en kullanıslı olan Model 3'tür

    Model 1:R-squared:0.622  Adj. R-squared:0.620  F-statistic:265.2 Prob(F-statistic):6.58e-299  AIC:2742 BIC:2795
		
    Model 2:R-squared:0.678 Adj. R-squared:0.676    F-statistic:277.3    Prob(F-statistic):0.00   AIC:2512    BIC:2576
		
    Model 3:R-squared:0.806 Adj. R-squared:0.804   F-statistic:462.5     Prob(F-statistic):0.00   AIC:1776    BIC:1850
		
